### Using phonorm

This notebook shows you how you can load pre-trained phonorm models and use them to predict the pronunciation of words

In [1]:
# Set the working directory
# We assume that you are in the '../examples' directory
import os
path = os.getcwd().replace('\\examples', '')
os.chdir(path)

In [2]:
# Load the functions necessary to process the data
from phonorm.utilities import create_mapping, tensor_from_pair, one_hot_encode, decode_from_ohe
from phonorm.tests import check_ohe
from phonorm.evaluate import plot_model_history, decode_sequence, evaluate_bleu, plot_bleu
from phonorm.Seq2Seq import Seq2Seq
from keras.optimizers import Adam
import numpy as np
import random

Using TensorFlow backend.


We first load the `cmudict (single)` dataset and preprocess it. 

In [4]:
# Load preprocessed data
pairs = np.load("data/preprocessed/cmudict_singlechar_train.npy")

# Create mapping
input_lang, output_lang = create_mapping("input", "output", pairs)

# Sample training pairs (len(pairs) == entire training data set)
ml = len(pairs)
random.seed(245)
pairs_array = [random.choice(pairs) for i in range(ml)]

# Split array into input / output vectors
input_array = [pair[0] for pair in pairs_array]
output_array = [pair[1] for pair in pairs_array]

## One-hot encoding
encoder_in_ohe = one_hot_encode(input_array, input_lang)
decoder_in_ohe = one_hot_encode(output_array, output_lang)
decoder_out_ohe = one_hot_encode(output_array, output_lang, one_timestep_ahead=True)

In [5]:
## Check if one-hot encoding works properly by sampling 2 random pairs
check_ohe(pairs, input_lang, output_lang, 2)

decoded and 'target decoded' should be the same

---- Example 0
Input: crumb, decoded: crumb
Output: krahm, decoded: krahm, target decoded: krahm


---- Example 1
Input: caspers, decoded: caspers
Output: kaesperz, decoded: kaesperz, target decoded: kaesperz




Next, we set up the encoder/decoder structure

In [6]:
# Set up the encoder/decoder model
phonorm = Seq2Seq(512, input_lang, output_lang)

In [8]:
# Load the cmudict (single) model
phonorm.load("models/cmudict/singlechar_model_10EP_H512")

In [9]:
# You can now use the predict() function 
phonorm.predict("aaron") # The first run takes a while

'ehrahn'

In [16]:
sentence = 'Donald Trump went to North Korea today to talk about the denuclearisation of the Korean peninsula'

In [23]:
sentence_out = [phonorm.predict(word.lower()) for word in sentence.split(" ")]

In [24]:
" ".join(sentence_out)

'daanahld trahmp wehnt tuw naorth kaoriyah tahdey tuw taok ahbawt dhey diynuwklerehseyshahn aof dhey kaoriyahn pehnihnsuwlah'